In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go



In [640]:

# 1. Cargar los datos
df = pd.read_csv('../data/training_pruebas.csv', encoding='latin1', sep=';', on_bad_lines='skip')
print(df.shape)

# 2. Verificar los valores nulos
print("Valores nulos por columna antes de eliminar:")
print(df.isnull().sum())

# 3. Definir un umbral para eliminar columnas con demasiados valores nulos
threshold = 0.5  # 50% de nulos

# 4. Almacenar el tamaño inicial del DataFrame
initial_size = df.shape

# 5. Eliminar columnas que tienen más del umbral definido de datos nulos
columns_to_remove = df.columns[df.isnull().mean() > threshold]

# Eliminar columnas y mostrar el tamaño del DataFrame después de cada eliminación
for column in columns_to_remove:
    df.drop(column, axis=1, inplace=True)
    print(f"Eliminada columna: {column}")
    print(f"Tamaño del dataset: {df.shape}")

# 6. Imprimir el tamaño final del DataFrame
print(f"Tamaño del dataset después de eliminar columnas: {df.shape}")

# 7. Imprimir el número de valores nulos por columna después de la eliminación
print("Valores nulos por columna después de eliminar:")
print(df.isnull().sum())

# 8. Llenar los valores nulos en columnas numéricas con la media de cada columna
numeric_cols = df.select_dtypes(include=np.number).columns
for col in numeric_cols:
    mean_value = df[col].mean()
    df[col].fillna(mean_value, inplace=True)  # Modifica el DataFrame directamente
    print(f"Llenada columna '{col}' con la media: {mean_value}")

# 9. Verificar que ya no hay valores nulos
print("Valores nulos por columna después de llenar con la media:")

print(df.isnull().sum())


C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_2752\2857641878.py:2: DtypeWarning:

Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.



(111430, 78)
Valores nulos por columna antes de eliminar:
ESTU_TIPODOCUMENTO             0
ESTU_NACIONALIDAD              0
ESTU_GENERO                    0
ESTU_FECHANACIMIENTO           2
ESTU_EXTERIOR                  0
                            ... 
PUNT_GLOBAL                    0
ESTU_INSE_INDIVIDUAL        3925
ESTU_NSE_INDIVIDUAL         3925
ESTU_NSE_IES                   0
ESTU_ESTADOINVESTIGACION       0
Length: 78, dtype: int64
Eliminada columna: ESTU_CURSODOCENTESIES
Tamaño del dataset: (111430, 77)
Eliminada columna: ESTU_CURSOIESAPOYOEXTERNO
Tamaño del dataset: (111430, 76)
Eliminada columna: ESTU_CURSOIESEXTERNA
Tamaño del dataset: (111430, 75)
Eliminada columna: ESTU_SIMULACROTIPOICFES
Tamaño del dataset: (111430, 74)
Eliminada columna: ESTU_ACTIVIDADREFUERZOAREAS
Tamaño del dataset: (111430, 73)
Eliminada columna: ESTU_ACTIVIDADREFUERZOGENERIC
Tamaño del dataset: (111430, 72)
Eliminada columna: ESTU_PRESENTACIONCASA
Tamaño del dataset: (111430, 71)
Tamaño del datase

In [641]:

# Limpiar la columna 'FAMI_ESTRATOVIVIENDA'
df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '').str.strip()
df['FAMI_ESTRATOVIVIENDA'] = pd.to_numeric(df['FAMI_ESTRATOVIVIENDA'], errors='coerce')

# Calcular la media de los valores disponibles en la columna y redondear al entero más cercano
mean_value = round(df['FAMI_ESTRATOVIVIENDA'].mean())

# Reemplazar NaN con la media calculada (ya redondeada)
df['FAMI_ESTRATOVIVIENDA'].fillna(mean_value, inplace=True)

# Convertir la columna a tipo entero
df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].astype(int)

In [642]:

# 1. Crear una variable que combine la educación de ambos padres
df['FAMI_EDUCACION_PADRES_COMBINADA'] = df['FAMI_EDUCACIONPADRE'] + df['FAMI_EDUCACIONMADRE']

# 2. Crear una variable binaria para acceso a tecnología
df['ACCESO_TECNOLOGIA'] = ((df['FAMI_TIENECOMPUTADOR'] == 'Si') & (df['FAMI_TIENEINTERNET'] == 'Si')).astype(int)

# 3. Crear una variable categórica para el nivel de trabajo del estudiante
def categorizar_horas_trabajo(horas):
    if horas == '0':
        return 'No trabaja'
    elif horas == 'Menos de 10 horas':
        return 'Tiempo parcial bajo'
    elif horas in ['Entre 11 y 20 horas', 'Entre 21 y 30 horas']:
        return 'Tiempo parcial alto'
    else:
        return 'Tiempo completo'

df['NIVEL_TRABAJO_ESTUDIANTE'] = df['ESTU_HORASSEMANATRABAJA'].apply(categorizar_horas_trabajo)

# 4. Crear una variable que indique si el estudiante se preparó para el examen
df['PREPARO_EXAMEN'] = (df['ESTU_COMOCAPACITOEXAMENSB11'] != 'No realizó ninguna prueba de preparación').astype(int)

# 5. Crear una variable que combine el estrato socioeconómico con el pago de matrícula
def combinar_estrato_matricula(estrato, matricula):
    if pd.isna(estrato) or pd.isna(matricula):
        return np.nan
    return f"{estrato}_{matricula}"

df['ESTRATO_MATRICULA'] = df.apply(lambda row: combinar_estrato_matricula(row['FAMI_ESTRATOVIVIENDA'], row['ESTU_VALORMATRICULAUNIVERSIDAD']), axis=1)

# 6. Crear una variable que indique si el estudiante es de primera generación en la universidad
df['PRIMERA_GENERACION_UNIV'] = ((df['FAMI_EDUCACIONPADRE'] != 'Educación profesional completa') & 
                                 (df['FAMI_EDUCACIONMADRE'] != 'Educación profesional completa')).astype(int)

# 7. Crear una variable que combine el método del programa y el carácter de la institución
df['METODO_CARACTER_COMBINADO'] = df['ESTU_METODO_PRGM'] + '_' + df['INST_CARACTER_ACADEMICO']

# 8. Crear una variable que indique si el estudiante reside en el mismo municipio donde estudia
df['RESIDE_DONDE_ESTUDIA'] = (df['ESTU_MCPIO_RESIDE'] == df['ESTU_PRGM_MUNICIPIO']).astype(int)

print(df[['FAMI_EDUCACION_PADRES_COMBINADA', 'ACCESO_TECNOLOGIA', 'NIVEL_TRABAJO_ESTUDIANTE', 
          'PREPARO_EXAMEN', 'ESTRATO_MATRICULA', 'PRIMERA_GENERACION_UNIV', 
          'METODO_CARACTER_COMBINADO', 'RESIDE_DONDE_ESTUDIA']].dtypes)


FAMI_EDUCACION_PADRES_COMBINADA    object
ACCESO_TECNOLOGIA                   int32
NIVEL_TRABAJO_ESTUDIANTE           object
PREPARO_EXAMEN                      int32
ESTRATO_MATRICULA                  object
PRIMERA_GENERACION_UNIV             int32
METODO_CARACTER_COMBINADO          object
RESIDE_DONDE_ESTUDIA                int32
dtype: object


In [643]:
# 1. Crear variables dummy para columnas categóricas
categorical_columns = ['FAMI_EDUCACION_PADRES_COMBINADA', 'NIVEL_TRABAJO_ESTUDIANTE', 
                       'ESTRATO_MATRICULA', 'METODO_CARACTER_COMBINADO']

# Crear dummies
df_dummies = pd.get_dummies(df[categorical_columns], drop_first=True)

# 2. Calcular la correlación de todas las variables con PUNT_GLOBAL
correlations = df.corr()['PUNT_GLOBAL'].sort_values(key=abs, ascending=False)

# 3. Filtrar las columnas dummy que tienen correlaciones mayores que 0.24
high_correlation_vars = correlations[correlations.abs() > 0.1].index.tolist()

# 4. Unir solo las variables que cumplen la condición
df_high_corr = pd.concat([df[high_correlation_vars], df_dummies], axis=1)

# 5. Mostrar las nuevas correlaciones con PUNT_GLOBAL en el nuevo DataFrame
new_correlations = df_high_corr.corr()['PUNT_GLOBAL'].sort_values(key=abs, ascending=False)

# 6. Mostrar las correlaciones
print("\nCorrelaciones con PUNT_GLOBAL (de mayor a menor):")
print(new_correlations)


C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_2752\2943024771.py:9: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.




Correlaciones con PUNT_GLOBAL (de mayor a menor):
PUNT_GLOBAL                                                                    1.000000
METODO_CARACTER_COMBINADO_PRESENCIAL_UNIVERSIDAD                               0.345233
ESTU_NSE_IES                                                                   0.262818
FAMI_ESTRATOVIVIENDA                                                           0.257611
ESTU_SNIES_PRGMACADEMICO                                                      -0.232677
                                                                                 ...   
ESTRATO_MATRICULA_5_Menos de 500 mil                                          -0.000461
FAMI_EDUCACION_PADRES_COMBINADA_No sabeSecundaria (Bachillerato) incompleta    0.000358
FAMI_EDUCACION_PADRES_COMBINADA_No AplicaTécnica o tecnológica incompleta     -0.000222
FAMI_EDUCACION_PADRES_COMBINADA_Técnica o tecnológica incompletaNo sabe        0.000117
FAMI_EDUCACION_PADRES_COMBINADA_Educación profesional incompletaNo sa

In [644]:
# Convertir la columna 'ESTU_FECHANACIMIENTO' a formato de fecha
from datetime import datetime

df['ESTU_FECHANACIMIENTO'] = pd.to_datetime(df['ESTU_FECHANACIMIENTO'], format='%d/%m/%Y', errors='coerce')
# Obtener la fecha actual
fecha_actual = pd.to_datetime(datetime.today())


# Calcular la edad en años
df['EDAD'] = (fecha_actual - df['ESTU_FECHANACIMIENTO']).dt.days // 365
df.drop(columns='ESTU_FECHANACIMIENTO', inplace=True)


mean_value = df['EDAD'].mean()
df['EDAD'].fillna(mean_value, inplace=True)  # Modifica el DataFrame directamente
print(f"Llenada columna '{col}' con la media: {mean_value}")

print(df['EDAD'].head())



Llenada columna 'ESTU_NSE_IES' con la media: 28.3889576938955
0    23.0
1    23.0
2    27.0
3    29.0
4    46.0
Name: EDAD, dtype: float64


In [645]:
# 1. Interacción entre acceso a tecnología y método de estudio
df['TECNOLOGIA_METODO'] = df['ACCESO_TECNOLOGIA'] * pd.get_dummies(df['ESTU_METODO_PRGM'])['PRESENCIAL']

# 2. Combinar nivel de trabajo del estudiante con preparación para el examen
df['TRABAJO_PREPARACION'] = df['NIVEL_TRABAJO_ESTUDIANTE'].astype(str) + '_' + df['PREPARO_EXAMEN'].astype(str)
df = pd.get_dummies(df, columns=['TRABAJO_PREPARACION'], drop_first=True)

# Calcular correlaciones de las nuevas variables numéricas con PUNT_GLOBAL
# Filtrar solo las variables numéricas
numerical_columns = df.select_dtypes(include=['number']).columns
correlations = df[numerical_columns].corr()['PUNT_GLOBAL'].sort_values(key=abs, ascending=False)

# Imprimir las correlaciones de mayor a menor
print("\nCorrelaciones de las nuevas variables combinadas con PUNT_GLOBAL (de mayor a menor):")
print(correlations)



Correlaciones de las nuevas variables combinadas con PUNT_GLOBAL (de mayor a menor):
PUNT_GLOBAL                                  1.000000
TECNOLOGIA_METODO                            0.269122
ESTU_NSE_IES                                 0.262818
FAMI_ESTRATOVIVIENDA                         0.257611
EDAD                                        -0.253269
ESTU_SNIES_PRGMACADEMICO                    -0.232677
ESTU_NSE_INDIVIDUAL                          0.215721
INST_COD_INSTITUCION                        -0.205356
ACCESO_TECNOLOGIA                            0.167829
PRIMERA_GENERACION_UNIV                     -0.166364
RESIDE_DONDE_ESTUDIA                         0.155606
ESTU_COD_MCPIO_PRESENTACION                 -0.114342
ESTU_COD_DEPTO_PRESENTACION                 -0.114010
ESTU_INSE_INDIVIDUAL                         0.112723
ESTU_COD_RESIDE_MCPIO                       -0.102957
PREPARO_EXAMEN                              -0.091734
TRABAJO_PREPARACION_No trabaja_1             0.085

In [646]:
# Filtrar las columnas categóricas
categorical_cols = df.select_dtypes(include=np.object_).columns

# Contar y mostrar la cantidad de valores únicos en cada columna categórica
for column in categorical_cols:
    unique_count = df[column].nunique()
    print(f"Cantidad de valores únicos en la columna '{column}': {unique_count}")
print(df.shape)

Cantidad de valores únicos en la columna 'ESTU_TIPODOCUMENTO': 8
Cantidad de valores únicos en la columna 'ESTU_NACIONALIDAD': 30
Cantidad de valores únicos en la columna 'ESTU_GENERO': 2
Cantidad de valores únicos en la columna 'ESTU_EXTERIOR': 2
Cantidad de valores únicos en la columna 'ESTU_CONSECUTIVO': 111430
Cantidad de valores únicos en la columna 'ESTU_ESTUDIANTE': 1
Cantidad de valores únicos en la columna 'ESTU_PAIS_RESIDE': 30
Cantidad de valores únicos en la columna 'ESTU_DEPTO_RESIDE': 46
Cantidad de valores únicos en la columna 'ESTU_MCPIO_RESIDE': 988
Cantidad de valores únicos en la columna 'ESTU_AREARESIDE': 2
Cantidad de valores únicos en la columna 'ESTU_ESTADOCIVIL': 4
Cantidad de valores únicos en la columna 'ESTU_TITULOOBTENIDOBACHILLER': 3
Cantidad de valores únicos en la columna 'ESTU_VALORMATRICULAUNIVERSIDAD': 8
Cantidad de valores únicos en la columna 'ESTU_PAGOMATRICULABECA': 2
Cantidad de valores únicos en la columna 'ESTU_PAGOMATRICULACREDITO': 2
Cantidad 

In [669]:

# Almacenar columnas para eliminar
columns_to_remove = []

# Verificar las columnas categóricas y agregar a la lista si tienen un solo valor único
for column in categorical_cols:
    try:
        if df[column].nunique() <= 1 or df[column].nunique() >= 1000:  # Cambiado a <= para incluir también columnas vacías
            columns_to_remove.append(column)
    except KeyError as e:
        print(f"Error: La columna '{column}' no se encuentra en el DataFrame. Detalle: {e}")

# Eliminar columnas que tienen solo un valor único
if columns_to_remove:
    df.drop(columns=columns_to_remove, inplace=True)

    # Mostrar las columnas eliminadas y el tamaño del DataFrame después de la eliminación
    for column in columns_to_remove:
        print(f"Eliminada columna: {column}")

print(f"Tamaño del dataset después de eliminar columnas: {df.shape}")


Tamaño del dataset después de eliminar columnas: (111430, 117)


In [1]:
categorical_cols = df.select_dtypes(include=np.object_).columns

# Iterar sobre cada columna categórica
for column in categorical_cols:
    # Crear variables dummy
    dummies = pd.get_dummies(df[column], prefix='DUMMY', drop_first=True)
    
    # Calcular la correlación de las dummies con la variable objetivo
    correlation_with_target = dummies.corrwith(df['PUNT_GLOBAL'])
    
    # Filtrar dummies con correlación mayor a 0.15
    significant_dummies = correlation_with_target[correlation_with_target.abs() > 0.15]

    # Si hay dummies significativas, añadirlas al DataFrame original
    if not significant_dummies.empty:
        for dummy in significant_dummies.index:
            # Añadir la columna dummy al DataFrame original
            df[dummy] = dummies[dummy]

# Mostrar las primeras filas del DataFrame final con las variables dummy añadidas
print("DataFrame final con las columnas dummy de alta correlación:")


NameError: name 'df' is not defined

In [ ]:


# Seleccionar solo las columnas numéricas
numerical_columns = df.select_dtypes(include=np.number).columns

# Crear un histograma para cada columna numérica
df[numerical_columns].hist(figsize=(25, 15), bins=15, edgecolor='black')

# Mostrar los histogramas
plt.tight_layout()
plt.show()

In [ ]:
for var in numerical_columns:
    if var != 'PUNT_GLOBAL':  # Evitar graficar 'sales_volume' contra sí mismo
        fig = px.scatter(df, x=var, y='PUNT_GLOBAL', title=f'{var} vs. PUNT_GLOBAL')
        fig.show()

In [649]:
fig = px.scatter(df, x='PUNT_GLOBAL', y=df['EDAD'], title= 'FAMI_ESTRATOVIVIENDA vs. PUNT_GLOBAL')
fig.show()

In [776]:
# Seleccionar solo las columnas numéricas
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()

# Calcular la correlación de 'PUNT_GLOBAL' con las demás variables numéricas
corr_with_punt_global = df[numerical_columns].corr()['PUNT_GLOBAL'].drop('PUNT_GLOBAL')

# Crear un DataFrame para el Heatmap
corr_df = pd.DataFrame(corr_with_punt_global).reset_index()
corr_df.columns = ['Variable', 'Correlación con PUNT_GLOBAL']

# Filtrar las correlaciones mayores a 0.2
filtered_corr_df = corr_df[abs(corr_df['Correlación con PUNT_GLOBAL']) >= 0.01]

# Crear el Heatmap
fig = go.Figure(data=go.Heatmap(
                z=filtered_corr_df['Correlación con PUNT_GLOBAL'].values.reshape(1, -1),
                x=filtered_corr_df['Variable'],
                y=['PUNT_GLOBAL'],
                colorscale='RdBu',
                zmin=-1, zmax=1,
                text=filtered_corr_df['Correlación con PUNT_GLOBAL'].values.reshape(1, -1),
                texttemplate='%{text:.2f}',
                textfont={"size":10},
                hoverongaps=False))

fig.update_layout(title='Correlación de Variables con PUNT_GLOBAL (mayor a 0.2)', width=2500, height=400)
fig.show()


In [ ]:
corr_with_punt_global = df[numerical_columns].corr()['PUNT_GLOBAL'].drop('PUNT_GLOBAL')
corr_with_punt_global.all()

In [ ]:
# Calcular la correlación de las variables numéricas
correlation_matrix = df.corr()

# Limitar las columnas a las que contienen algún dato numérico para un análisis más detallado
numerical_columns = df.select_dtypes(include=[np.number]).columns


# Filtrar la matriz de correlación
correlation_matrix_filtered = correlation_matrix.loc[numerical_columns, numerical_columns]

# Crear el Heatmap
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix_filtered.values,
    x=correlation_matrix_filtered.columns,
    y=correlation_matrix_filtered.columns,
    colorscale='RdBu',
    zmin=-1, zmax=1,
    text=correlation_matrix_filtered.values,
    texttemplate='%{text:.2f}',
    textfont={"size":10},
    hoverongaps=False))

fig.update_layout(title='Matriz de Correlación', width=1800, height=1800)
fig.show()

In [796]:

#ESTU_COD_MCPIO_PRESENTACION
#ESTU_NSE_INDIVIDUAL
dk = df[filtered_corr_df['Variable']]

dk = dk.drop(columns=['DUMMY_PARIS'])

len(dk.columns),dk.shape


(410, (111430, 410))

In [797]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import pandas as pd

# Suponiendo que 'dk' es tu DataFrame
X = dk  # Otras características
y = df['PUNT_GLOBAL']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# No se escalan los datos

params = {
'n_estimators': 400,
    'max_depth': 16,
    'min_samples_split': 50,
    'min_samples_leaf': 10,
    'learning_rate': 0.04,
    'max_features': 'sqrt'
}

gbm = GradientBoostingRegressor(random_state=42, **params)

# Entrenar el modelo
gbm.fit(X_train, y_train)




GradientBoostingRegressor(learning_rate=0.04, max_depth=16, max_features='sqrt',
                          min_samples_leaf=10, min_samples_split=50,
                          n_estimators=400, random_state=42)

In [798]:

# Predecir en el conjunto de entrenamiento y prueba
y_train_pred = gbm.predict(X_train)
y_test_pred = gbm.predict(X_test)

# Calcular MSE y R² para el conjunto de entrenamiento
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Calcular MSE y R² para el conjunto de prueba
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Imprimir los resultados
print(f"Train MSE: {mse_train}")
print(f"Train R²: {r2_train}")
print(f"Test MSE: {mse_test}")
print(f"Test R²: {r2_test}")

"""Train MSE: 199.5897064435297
Train R²: 0.6366506888025377
Test MSE: 326.31638332608634
Test R²: 0.41076512770502116

'n_estimators': 400,
    'max_depth': 16,
    'min_samples_split': 50,
    'min_samples_leaf': 10,
    'learning_rate': 0.04,
    'max_features': 'sqrt',
"""

Train MSE: 199.7318265648892
Train R²: 0.6363919617913936
Test MSE: 326.56389755205134
Test R²: 0.41031818718722946


"Train MSE: 199.5897064435297\nTrain R²: 0.6366506888025377\nTest MSE: 326.31638332608634\nTest R²: 0.41076512770502116\n\n'n_estimators': 400,\n    'max_depth': 16,\n    'min_samples_split': 50,\n    'min_samples_leaf': 10,\n    'learning_rate': 0.04,\n    'max_features': 'sqrt',\n"

In [677]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

# Combinar X (dk) y y (df['PUNT_GLOBAL']) en un único DataFrame temporal
df_combined = pd.concat([dk, df['PUNT_GLOBAL']], axis=1)

# Eliminar las filas con valores nulos en cualquiera de las variables
df_combined = df_combined.dropna()

# Separar de nuevo en variables independientes (X) y la variable objetivo (y)
X = df_combined.drop('PUNT_GLOBAL', axis=1)  # Eliminar la columna objetivo
y = df_combined['PUNT_GLOBAL']  # La columna objetivo
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un pipeline para preprocesamiento y modelado
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('ridge', Ridge(alpha=10))
])

# Entrenar el modelo
pipeline.fit(X_train, y_train)



MemoryError: Unable to allocate 16.5 GiB for an array with shape (89144, 24803) and data type float64

In [676]:


# Hacer predicciones
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

# Calcular métricas de rendimiento
def print_metrics(y_true, y_pred, set_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f'Métricas para el conjunto {set_name}:')
    print(f'MSE: {mse:.4f}')
    print(f'R2: {r2:.4f}\n')

print_metrics(y_train, y_train_pred, 'de entrenamiento')
print_metrics(y_test, y_test_pred, 'de prueba')

Métricas para el conjunto de entrenamiento:
MSE: 370.0259
R2: 0.3264

Métricas para el conjunto de prueba:
MSE: 382.3483
R2: 0.3096



In [678]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.neighbors import KNeighborsRegressor

# Combinar X (dk) y y (df['PUNT_GLOBAL']) en un único DataFrame temporal
df_combined = pd.concat([dk, df['PUNT_GLOBAL']], axis=1)

# Eliminar las filas con valores nulos en cualquiera de las variables
df_combined = df_combined.dropna()

# Separar de nuevo en variables independientes (X) y la variable objetivo (y)
X = df_combined.drop('PUNT_GLOBAL', axis=1)  # Eliminar la columna objetivo
y = df_combined['PUNT_GLOBAL']  # La columna objetivo

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Crear el modelo KNN (n_neighbors determina cuántos vecinos usar)
knn = KNeighborsRegressor(n_neighbors=20)

# Entrenar el modelo
knn.fit(X_train, y_train)



KNeighborsRegressor(n_neighbors=20)

In [679]:
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# Calcular métricas de rendimiento
def print_metrics(y_true, y_pred, set_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'Métricas para el conjunto {set_name}:')
    print(f'MSE: {mse:.4f}')
    print(f'R2: {r2:.4f}')
    print(f'MAPE: {mape:.4f}\n')

print_metrics(y_train, y_train_pred, 'de entrenamiento')
print_metrics(y_test, y_test_pred, 'de prueba')

Métricas para el conjunto de entrenamiento:
MSE: 485.0480
R2: 0.1170
MAPE: 7401253538205.4746

Métricas para el conjunto de prueba:
MSE: 536.8405
R2: 0.0306
MAPE: 29655534654788.8008



In [74]:
dl = pd.read_csv('../data/test_pruebas.csv', encoding='latin1', sep=';', on_bad_lines='skip')

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_2752\1438923777.py:1: DtypeWarning:

Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.



In [76]:
id = dl['ESTU_CONSECUTIVO']

In [77]:
# 2. Verificar los valores nulos
print("Valores nulos por columna antes de eliminar:")
print(dl.isnull().sum())

# 7. Imprimir el número de valores nulos por columna después de la eliminación
print("Valores nulos por columna después de eliminar:")
print(dl.isnull().sum())

# 8. Llenar los valores nulos en columnas numéricas con la media de cada columna
numeric_cols = dl.select_dtypes(include=np.number).columns
for col in numeric_cols:
    mean_value = dl[col].mean()
    dl[col].fillna(mean_value, inplace=True)  # Modifica el DataFrame directamente
    print(f"Llenada columna '{col}' con la media: {mean_value}")

# 9. Verificar que ya no hay valores nulos
print("Valores nulos por columna después de llenar con la media:")

print(dl.isnull().sum())


Valores nulos por columna antes de eliminar:
ESTU_TIPODOCUMENTO               0
ESTU_NACIONALIDAD                0
ESTU_GENERO                      0
ESTU_FECHANACIMIENTO             1
ESTU_EXTERIOR                    0
                              ... 
ESTU_COD_DEPTO_PRESENTACION      0
ESTU_INSE_INDIVIDUAL           970
ESTU_NSE_INDIVIDUAL            970
ESTU_NSE_IES                     0
ESTU_ESTADOINVESTIGACION         0
Length: 77, dtype: int64
Valores nulos por columna después de eliminar:
ESTU_TIPODOCUMENTO               0
ESTU_NACIONALIDAD                0
ESTU_GENERO                      0
ESTU_FECHANACIMIENTO             1
ESTU_EXTERIOR                    0
                              ... 
ESTU_COD_DEPTO_PRESENTACION      0
ESTU_INSE_INDIVIDUAL           970
ESTU_NSE_INDIVIDUAL            970
ESTU_NSE_IES                     0
ESTU_ESTADOINVESTIGACION         0
Length: 77, dtype: int64
Llenada columna 'PERIODO' con la media: 20231.014717495873
Llenada columna 'ESTU_COD_RES

In [78]:

# Limpiar la columna 'FAMI_ESTRATOVIVIENDA'
dl['FAMI_ESTRATOVIVIENDA'] = dl['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '').str.strip()
dl['FAMI_ESTRATOVIVIENDA'] = pd.to_numeric(dl['FAMI_ESTRATOVIVIENDA'], errors='coerce')

# Calcular la media de los valores disponibles en la columna y redondear al entero más cercano
mean_value = round(dl['FAMI_ESTRATOVIVIENDA'].mean())

# Reemplazar NaN con la media calculada (ya redondeada)
dl['FAMI_ESTRATOVIVIENDA'].fillna(mean_value, inplace=True)

# Convertir la columna a tipo entero
dl['FAMI_ESTRATOVIVIENDA'] = dl['FAMI_ESTRATOVIVIENDA'].astype(int)

In [79]:

# 1. Crear una variable que combine la educación de ambos padres
dl['FAMI_EDUCACION_PADRES_COMBINADA'] = dl['FAMI_EDUCACIONPADRE'] + dl['FAMI_EDUCACIONMADRE']

# 2. Crear una variable binaria para acceso a tecnología
dl['ACCESO_TECNOLOGIA'] = ((dl['FAMI_TIENECOMPUTADOR'] == 'Si') & (dl['FAMI_TIENEINTERNET'] == 'Si')).astype(int)

# 3. Crear una variable categórica para el nivel de trabajo del estudiante
def categorizar_horas_trabajo(horas):
    if horas == '0':
        return 'No trabaja'
    elif horas == 'Menos de 10 horas':
        return 'Tiempo parcial bajo'
    elif horas in ['Entre 11 y 20 horas', 'Entre 21 y 30 horas']:
        return 'Tiempo parcial alto'
    else:
        return 'Tiempo completo'

dl['NIVEL_TRABAJO_ESTUDIANTE'] = dl['ESTU_HORASSEMANATRABAJA'].apply(categorizar_horas_trabajo)

# 4. Crear una variable que indique si el estudiante se preparó para el examen
dl['PREPARO_EXAMEN'] = (dl['ESTU_COMOCAPACITOEXAMENSB11'] != 'No realizó ninguna prueba de preparación').astype(int)

# 5. Crear una variable que combine el estrato socioeconómico con el pago de matrícula
def combinar_estrato_matricula(estrato, matricula):
    if pd.isna(estrato) or pd.isna(matricula):
        return np.nan
    return f"{estrato}_{matricula}"

dl['ESTRATO_MATRICULA'] = dl.apply(lambda row: combinar_estrato_matricula(row['FAMI_ESTRATOVIVIENDA'], row['ESTU_VALORMATRICULAUNIVERSIDAD']), axis=1)

# 6. Crear una variable que indique si el estudiante es de primera generación en la universidad
dl['PRIMERA_GENERACION_UNIV'] = ((dl['FAMI_EDUCACIONPADRE'] != 'Educación profesional completa') & 
                                 (dl['FAMI_EDUCACIONMADRE'] != 'Educación profesional completa')).astype(int)

# 7. Crear una variable que combine el método del programa y el carácter de la institución
dl['METODO_CARACTER_COMBINADO'] = dl['ESTU_METODO_PRGM'] + '_' + dl['INST_CARACTER_ACADEMICO']

# 8. Crear una variable que indique si el estudiante reside en el mismo municipio donde estudia
dl['RESIDE_DONDE_ESTUDIA'] = (dl['ESTU_MCPIO_RESIDE'] == dl['ESTU_PRGM_MUNICIPIO']).astype(int)

print(dl[['FAMI_EDUCACION_PADRES_COMBINADA', 'ACCESO_TECNOLOGIA', 'NIVEL_TRABAJO_ESTUDIANTE', 
          'PREPARO_EXAMEN', 'ESTRATO_MATRICULA', 'PRIMERA_GENERACION_UNIV', 
          'METODO_CARACTER_COMBINADO', 'RESIDE_DONDE_ESTUDIA']].dtypes)


FAMI_EDUCACION_PADRES_COMBINADA    object
ACCESO_TECNOLOGIA                   int32
NIVEL_TRABAJO_ESTUDIANTE           object
PREPARO_EXAMEN                      int32
ESTRATO_MATRICULA                  object
PRIMERA_GENERACION_UNIV             int32
METODO_CARACTER_COMBINADO          object
RESIDE_DONDE_ESTUDIA                int32
dtype: object


In [81]:
dl['ESTU_FECHANACIMIENTO']

0        11/04/1975
1        17/05/1998
2        11/01/1998
3        16/04/1994
4        16/07/1993
            ...    
27853    28/01/1992
27854     3/04/2001
27855     3/07/1971
27856    30/05/1996
27857    11/10/2001
Name: ESTU_FECHANACIMIENTO, Length: 27858, dtype: object

In [82]:
# Convertir la columna 'ESTU_FECHANACIMIENTO' a formato de fecha
from datetime import datetime

dl['ESTU_FECHANACIMIENTO'] = pd.to_datetime(dl['ESTU_FECHANACIMIENTO'], format='%d/%m/%Y', errors='coerce')
# Obtener la fecha actual
fecha_actual = pd.to_datetime(datetime.today())


# Calcular la edad en años
dl['EDAD'] = (fecha_actual - dl['ESTU_FECHANACIMIENTO']).dt.days // 365
dl.drop(columns='ESTU_FECHANACIMIENTO', inplace=True)


mean_value = dl['EDAD'].mean()
dl['EDAD'].fillna(mean_value, inplace=True)  # Modifica el DataFrame directamente
print(f"Llenada columna '{col}' con la media: {mean_value}")

dl['EDAD']



Llenada columna 'ESTU_NSE_IES' con la media: 28.422981656316185


0        49.0
1        26.0
2        26.0
3        30.0
4        31.0
         ... 
27853    32.0
27854    23.0
27855    53.0
27856    28.0
27857    23.0
Name: EDAD, Length: 27858, dtype: float64

In [83]:
# 1. Interacción entre acceso a tecnología y método de estudio
dl['TECNOLOGIA_METODO'] = dl['ACCESO_TECNOLOGIA'] * pd.get_dummies(dl['ESTU_METODO_PRGM'])['PRESENCIAL']

# 2. Combinar nivel de trabajo del estudiante con preparación para el examen
dl['TRABAJO_PREPARACION'] = dl['NIVEL_TRABAJO_ESTUDIANTE'].astype(str) + '_' + dl['PREPARO_EXAMEN'].astype(str)
dl = pd.get_dummies(dl, columns=['TRABAJO_PREPARACION'], drop_first=True)


In [84]:
# Filtrar las columnas categóricas
categorical_cols = dl.select_dtypes(include=np.object_).columns

# Contar y mostrar la cantidad de valores únicos en cada columna categórica
for column in categorical_cols:
    unique_count = dl[column].nunique()
    print(f"Cantidad de valores únicos en la columna '{column}': {unique_count}")
print(dl.shape)

Cantidad de valores únicos en la columna 'ESTU_TIPODOCUMENTO': 8
Cantidad de valores únicos en la columna 'ESTU_NACIONALIDAD': 13
Cantidad de valores únicos en la columna 'ESTU_GENERO': 2
Cantidad de valores únicos en la columna 'ESTU_EXTERIOR': 2
Cantidad de valores únicos en la columna 'ESTU_CONSECUTIVO': 27858
Cantidad de valores únicos en la columna 'ESTU_ESTUDIANTE': 1
Cantidad de valores únicos en la columna 'ESTU_PAIS_RESIDE': 13
Cantidad de valores únicos en la columna 'ESTU_DEPTO_RESIDE': 36
Cantidad de valores únicos en la columna 'ESTU_MCPIO_RESIDE': 812
Cantidad de valores únicos en la columna 'ESTU_AREARESIDE': 2
Cantidad de valores únicos en la columna 'ESTU_ESTADOCIVIL': 4
Cantidad de valores únicos en la columna 'ESTU_TITULOOBTENIDOBACHILLER': 3
Cantidad de valores únicos en la columna 'ESTU_VALORMATRICULAUNIVERSIDAD': 8
Cantidad de valores únicos en la columna 'ESTU_PAGOMATRICULABECA': 2
Cantidad de valores únicos en la columna 'ESTU_PAGOMATRICULACREDITO': 2
Cantidad d

In [85]:

# Almacenar columnas para eliminar
columns_to_remove = []

# Verificar las columnas categóricas y agregar a la lista si tienen un solo valor único
for column in categorical_cols:
    try:
        if dl[column].nunique() <= 1 or dl[column].nunique() >= 1000:  # Cambiado a <= para incluir también columnas vacías
            columns_to_remove.append(column)
    except KeyError as e:
        print(f"Error: La columna '{column}' no se encuentra en el DataFrame. Detalle: {e}")

# Eliminar columnas que tienen solo un valor único
if columns_to_remove:
    dl.drop(columns=columns_to_remove, inplace=True)

    # Mostrar las columnas eliminadas y el tamaño del DataFrame después de la eliminación
    for column in columns_to_remove:
        print(f"Eliminada columna: {column}")

print(f"Tamaño del dataset después de eliminar columnas: {dl.shape}")
len(dl.columns)


Eliminada columna: ESTU_CONSECUTIVO
Eliminada columna: ESTU_ESTUDIANTE
Eliminada columna: ESTU_NIVEL_PRGM_ACADEMICO
Eliminada columna: ESTU_PRIVADO_LIBERTAD
Eliminada columna: ESTU_ESTADOINVESTIGACION
Tamaño del dataset después de eliminar columnas: (27858, 88)


88

In [86]:
# Seleccionar las columnas categóricas
categorical_col = dl.select_dtypes(include='object').columns

# Iterar sobre cada columna categórica
for column in categorical_col:
    # Crear variables dummy para cada columna categórica con un prefijo basado en el nombre de la columna
    dummies = pd.get_dummies(dl[column], prefix='DUMMY', drop_first=True)
    
    # Añadir las variables dummy al DataFrame original
    dl = pd.concat([dl, dummies], axis=1)

# Eliminar las columnas categóricas originales, ya que ahora están en formato dummy
dl.drop(columns=categorical_col, inplace=True)

# Mostrar las primeras filas del DataFrame con las variables dummy añadidas
print("DataFrame final con las columnas dummy añadidas:")



DataFrame final con las columnas dummy añadidas:
['PERIODO', 'ESTU_COD_RESIDE_DEPTO', 'ESTU_COD_RESIDE_MCPIO', 'FAMI_ESTRATOVIVIENDA', 'INST_COD_INSTITUCION', 'ESTU_SNIES_PRGMACADEMICO', 'ESTU_PRGM_CODMUNICIPIO', 'ESTU_INST_CODMUNICIPIO', 'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_DEPTO_PRESENTACION', 'ESTU_INSE_INDIVIDUAL', 'ESTU_NSE_INDIVIDUAL', 'ESTU_NSE_IES', 'ACCESO_TECNOLOGIA', 'PREPARO_EXAMEN', 'PRIMERA_GENERACION_UNIV', 'RESIDE_DONDE_ESTUDIA', 'EDAD', 'TECNOLOGIA_METODO', 'TRABAJO_PREPARACION_No trabaja_1', 'TRABAJO_PREPARACION_Tiempo completo_0', 'TRABAJO_PREPARACION_Tiempo completo_1', 'TRABAJO_PREPARACION_Tiempo parcial alto_0', 'TRABAJO_PREPARACION_Tiempo parcial alto_1', 'TRABAJO_PREPARACION_Tiempo parcial bajo_0', 'TRABAJO_PREPARACION_Tiempo parcial bajo_1', 'DUMMY_CE', 'DUMMY_CR', 'DUMMY_PC', 'DUMMY_PE', 'DUMMY_PEP', 'DUMMY_PPT', 'DUMMY_TI', 'DUMMY_CHILE', 'DUMMY_COLOMBIA', 'DUMMY_COREA DEL SUR', 'DUMMY_ECUADOR', 'DUMMY_ESPAÑA', 'DUMMY_FRANCIA', 'DUMMY_ITALIA', 'DUMMY_LÍBA

In [87]:
len(dl.columns)

2771

In [806]:
# Eliminar 'DUMMY_PARIS' de 'filtered_corr_df' antes de crear el DataFrame 'dp'
filtered_corr_df = filtered_corr_df[filtered_corr_df['Variable'] != 'DUMMY_PARIS']

# Crear el DataFrame 'dp' a partir de las columnas filtradas
dp = dl[filtered_corr_df['Variable']]

# Eliminar columnas duplicadas en 'dp'
dp = dp.loc[:, ~dp.columns.duplicated()]

# Verificar el número de columnas
len(dp.columns)



410

In [420]:
#REGRESION LINEAL

# Hacer predicciones en los datos imputados
y_test_pred = pipeline.predict(dp)

# Crear un DataFrame con las predicciones
predicciones_dm = pd.DataFrame({
    'ID': id,  # ESTE ES EL ID CONSECUTIO DEL ESTUDIANTE SE SACA ANTES DE SER ELIMINADO 
    'TARGET': y_test_pred
})

# Guardar las predicciones en un archivo CSV
output_file = 'Regression.csv'
predicciones_dm.to_csv(output_file, index=False)

print(f"Predicciones guardadas en el archivo: {output_file}")

Predicciones guardadas en el archivo: Regression.csv


In [421]:
# KNN

# Hacer predicciones en los datos imputados
y_test_pred = knn.predict(dp)

# Crear un DataFrame con las predicciones
predicciones_dm = pd.DataFrame({
    'id': id, # 'id'  identificadores 
    'TARGET': y_test_pred
})

output_file = 'Knn_20.csv'
predicciones_dm.to_csv(output_file, index=False)

print(f"Predicciones guardadas en el archivo: {output_file}")

Predicciones guardadas en el archivo: Knn_20.csv


In [807]:

y_pred = gbm.predict(dp)


# Crear un DataFrame con las predicciones
predicciones_dm = pd.DataFrame({
    'id': id, # 'id'  identificadores 
    'TARGET': y_pred
})

output_file = 'gbm.csv'
predicciones_dm.to_csv(output_file, index=False)

print(f"Predicciones guardadas en el archivo: {output_file}")

Predicciones guardadas en el archivo: gbm.csv
